In [ ]:
pip install hyperliquid-python-sdk

# Example on how to buy (spot)

In [11]:
# Import the json module to handle JSON files
import json

# Importing constants from a custom utility module, likely containing configuration data
from hyperliquid.utils import constants
# Importing another module that might be responsible for utility functions or setups
import example_utils

# Constant representing a trading pair
COIN = "PURR/USDC"

# Main function definition
def main():
    # Setting up and returning essential components such as address, info object, and exchange object using a function from example_utils
    address, info, exchange = example_utils.setup(base_url=constants.TESTNET_API_URL, skip_ws=True)

    # Fetching the user's state in the spot market and printing position information
    spot_user_state = info.spot_user_state(address)
    # Check if there are any balances in the spot user state and print them
    if len(spot_user_state["balances"]) > 0:
        print("spot balances:")
        for balance in spot_user_state["balances"]:
            # Printing each balance in a readable format using JSON dumps
            print(json.dumps(balance, indent=2))
    else:
        # Message displayed if there are no token balances
        print("no available token balances")

    # Placing an order with a very low price intending it to rest on the order book
    order_result = exchange.order(COIN, True, 1, 95.727, {"limit": {"tif": "Gtc"}})
    # Printing the result of the order placement
    print(order_result)

    # Checking the status of the order and querying it by order ID if the status is 'ok'
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            # Querying the order by its ID to get its current status
            order_status = info.query_order_by_oid(address, status["resting"]["oid"])
            # Printing the status of the queried order
            print("Order status by oid:", order_status)

    # If the order is successfully placed and resting, attempting to cancel it
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            # Cancelling the order using its ID
            cancel_result = exchange.cancel(COIN, status["resting"]["oid"])
            # Printing the result of the cancel operation
            print(cancel_result)


# Checking if this script is the main program and not being imported elsewhere, then calling main function
if __name__ == "__main__":
    main()


Running with account address: 0x2039B73AE49F18EC2Faa0fC9E56e6c898d6C51b9
Running with agent address: 0xD157b235dB1024a6377Cafa9ceDFa0C71EcE95b3
spot balances:
{
  "coin": "USDC",
  "hold": "0.0",
  "token": 0,
  "total": "1435.387439"
}
{
  "coin": "PURR",
  "hold": "0.0",
  "token": 1,
  "total": "124.0"
}
{
  "coin": "CHUTORO",
  "hold": "0.0",
  "token": 11,
  "total": "0.24"
}
{'status': 'ok', 'response': {'type': 'order', 'data': {'statuses': [{'resting': {'oid': 8019151984}}]}}}
Order status by oid: {'order': {'order': {'children': [], 'cloid': None, 'coin': 'PURR/USDC', 'isPositionTpsl': False, 'isTrigger': False, 'limitPx': '95.727', 'oid': 8019151984, 'orderType': 'Limit', 'origSz': '1.0', 'reduceOnly': False, 'side': 'B', 'sz': '1.0', 'tif': 'Gtc', 'timestamp': 1714293397211, 'triggerCondition': 'N/A', 'triggerPx': '0.0'}, 'status': 'open', 'statusTimestamp': 1714293397211}, 'status': 'order'}
{'status': 'ok', 'response': {'type': 'cancel', 'data': {'statuses': ['success']}}}

# Example on transfer funds between spot and perps

In [21]:
from hyperliquid.utils import constants
import example_utils


def main():
    address, info, exchange = example_utils.setup(constants.TESTNET_API_URL, skip_ws=True)

    # Fetching the user's state in the spot market and printing position information
    spot_user_state = info.spot_user_state(address)
    # Using list comprehension to filter out the dictionary
    print(spot_user_state)
    usdc_data = [item for item in spot_user_state["balances"] if item['coin'] == 'USDC']
    print("USDC at start in spot:", usdc_data)
    
    user_state = info.user_state(address)
    print("USDC at start in perps:", user_state["withdrawable"])
    # Check if there are any balances in the spot user state and print them
    if len(spot_user_state["balances"]) > 0:
        print("spot balances:")
        print(spot_user_state["balances"])
    
    # Transfer USDC from perp wallet to spot wallet
    transfer_result = exchange.user_spot_transfer(2, False)
    print(transfer_result)

    # Transfer 1.23 USDC from spot wallet to perp wallet
    transfer_result = exchange.user_spot_transfer(4, True)
    print(transfer_result)
    
    spot_user_state = info.spot_user_state(address)
    usdc_data = [item for item in spot_user_state["balances"] if item['coin'] == 'USDC']
    print("USDC at end in spot:", usdc_data)
    
    user_state = info.user_state(address)
    print("USDC at end in perps:", user_state["withdrawable"])

if __name__ == "__main__":
    main()

Running with account address: 0x2039B73AE49F18EC2Faa0fC9E56e6c898d6C51b9
Running with agent address: 0xD157b235dB1024a6377Cafa9ceDFa0C71EcE95b3
{'balances': [{'coin': 'USDC', 'hold': '0.0', 'token': 0, 'total': '1433.347439'}, {'coin': 'PURR', 'hold': '0.0', 'token': 1, 'total': '124.0'}, {'coin': 'CHUTORO', 'hold': '0.0', 'token': 11, 'total': '0.24'}]}
USDC at start in spot: [{'coin': 'USDC', 'hold': '0.0', 'token': 0, 'total': '1433.347439'}]
USDC at start in perps: 386.832092
spot balances:
[{'coin': 'USDC', 'hold': '0.0', 'token': 0, 'total': '1433.347439'}, {'coin': 'PURR', 'hold': '0.0', 'token': 1, 'total': '124.0'}, {'coin': 'CHUTORO', 'hold': '0.0', 'token': 11, 'total': '0.24'}]
{'status': 'ok', 'response': {'type': 'default'}}
{'status': 'ok', 'response': {'type': 'default'}}
USDC at end in spot: [{'coin': 'USDC', 'hold': '0.0', 'token': 0, 'total': '1431.347439'}]
USDC at end in perps: 388.832092


# Example how to 